HAA from
https://allocatesmartly.com/hybrid-asset-allocation/


1. At the close on the last trading day of the month, measure the momentum of US Treasury Inflation-Protected Securities (TIPS, ETF: TIP).

    The strategy uses TIPS as a “canary asset” to determine whether to allocate to either an offensive or defensive asset universe. Since 1971, the momentum of simulated TIPS would have been positive about 86% of the time, meaning the vast majority of the time we would be allocating to the offensive asset universe.
    
2. If TIPS momentum is positive, select the four assets with the highest momentum from the following offensive asset universe:
    
    US large caps (represented by SPY), US small caps (IWM), developed international stocks (EFA), emerging market stocks (EEM), US real estate (VNQ), commodities (PDBC), intermediate-term US Treasuries (IEF) and long-term US Treasuries (TLT)
    
    For each of those four assets, if momentum is positive, allocate 25% of the portfolio to the asset, otherwise allocate that portion of the portfolio to cash either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum (US T-Bills are used as a proxy for cash momentum) (*).
    
    This is what’s known as “dual momentum”. The asset must exhibit both positive momentum (aka “time series momentum”) as well as high momentum relative to competing assets (aka “cross-sectional momentum”).
    
    Note: Commodities and treasuries are generally considered defensive assets, so the offensive portfolio will still be diversified in many cases.

3. If TIPS momentum is negative, allocate the entire portfolio to either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum.

4. All positions are executed at the close. Hold all positions until the last the trading day of the following month. Rebalance monthly, even if there is no change signaled.


In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sr(ret, N=252):
    ret = ret[ret!=0]
    print(f'SR: {np.mean(ret) / np.std(ret) * np.sqrt(N)}')

def cagr(cpnl):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1]
    print(f'CAGR: {FV ** (1/t) - 1}')

In [ ]:
# offensive_universe = ['SPY', 'IWM', 'EFA', 'EEM', 'VNQ', 'DBC', 'TLT']

US_Equities = ['SPY', 'IWM']
Foreign_Equities = ['VEA', 'VWO']
Alternative_Assets = ['DBC', 'VNQ']
US_Bonds = ['IEF', 'TLT']

offensive_universe = US_Equities + Foreign_Equities + Alternative_Assets + US_Bonds

cash_universe = ['BIL', 'IEF']
canary_asset = ['TIP']

whole_universe = offensive_universe + cash_universe + canary_asset

In [ ]:
prices_df = yf.download(tickers=whole_universe,
                        end=datetime.now())


In [ ]:
prices_df = prices_df['Adj Close']

In [ ]:
prices_df

In [ ]:
px.line(prices_df.melt(ignore_index=False), y='value', color='variable', log_y=True) 

In [ ]:
class MOMENTUM():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        self.calc_monthly_close(X)
        ret1m = self.get_return(1)
        ret3m = self.get_return(3)
        ret6m = self.get_return(6)
        ret12m = self.get_return(12)
        return (ret1m + ret3m + ret6m + ret12m) / 2
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('M').last()
    
    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)


class MONTHLY_RETURNS():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('M').last()

    def transform(self, X):
        self.calc_monthly_close(X)
        return self.get_return(1).shift(-1)

    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)
    

class MONTHLY_PICKS():
    def __init__(self, offensive_universe, cash_universe, canary_asset) -> None:
        self.offensive_universe = offensive_universe
        self.cash_universe = cash_universe
        self.canary_asset = canary_asset
        self.regime = None
        
    def get_picks(self, X):
        self.get_regime(X)
        if self.regime:
            assets = self.get_offensive_assets(X)
        else:
            assets = self.get_defensive_assets(X)
        return assets
    
    def get_regime(self, X):
        if X[canary_asset].values[0] > 0:
            self.regime = True
        else:
            self.regime = False

    def get_defensive_assets(self, X):
        asset = X[self.cash_universe].melt()
        asset = asset[asset.value > 0]
        asset = asset.sort_values('value').tail(1).variable.tolist()
        return asset

    def get_offensive_assets(self, X):
        assets = X[self.offensive_universe].melt()
        assets = assets[assets.value > 0]
        assets = assets.sort_values('value').tail(4).variable.tolist()
        Cash_fraction = 4 - len(assets)
        if Cash_fraction > 0:
            defensive_asset = self.get_defensive_assets(X)
            assets = assets + defensive_asset * Cash_fraction
        return assets

In [ ]:
mom = MOMENTUM()
mom_df = pd.concat([mom.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)

monthly_ret = MONTHLY_RETURNS()
ret_df = pd.concat([monthly_ret.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)


In [ ]:
px.line(mom_df.melt(ignore_index=False), y='value', color='variable')

In [ ]:
monthly_picks = MONTHLY_PICKS(offensive_universe, cash_universe, canary_asset)
monthly_assets = mom_df.dropna().groupby('Date').apply(monthly_picks.get_picks)

In [ ]:
months_ret = []

for Date in monthly_assets.index.unique():
    assets = monthly_assets.loc[Date] #.tolist()
    months_ret.append([Date, ret_df.loc[Date, assets].mean()])


In [ ]:
strat_returns = pd.DataFrame(months_ret, columns=['Date', 'ret']).sort_values('Date').set_index('Date')
strat_returns['cpnl'] = (strat_returns.ret + 1).cumprod()

In [ ]:
px.line(strat_returns.cpnl, log_y=True)

In [ ]:
cagr(strat_returns.cpnl.dropna())
sr(strat_returns.ret.dropna(), N=12)

In [ ]:
sr